https://cs230-stanford.github.io/tensorflow-input-data.html

In [1]:
import tensorflow as tf
tf.__version__

'1.10.0'

In [2]:
# tf.enable_eager_execution()

In [40]:
dataset = tf.data.TextLineDataset("file.txt")

In [4]:
for line in dataset:
    print(line)

RuntimeError: dataset.__iter__() is only supported when eager execution is enabled.

In [38]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [46]:
with tf.Session() as sess:
    for i in range(2):
        print(sess.run(next_element))

[[b'Both' b'are' b'great']
 [b'You' b'use' b'PyTorch']]
[[b'I' b'use' b'Tensorflow']]


In [41]:
dataset = dataset.map(lambda string: tf.string_split([string]).values)

In [42]:
dataset = dataset.shuffle(buffer_size=3)

In [43]:
dataset = dataset.batch(2)

In [44]:
dataset = dataset.prefetch(1)

In [45]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    print(sess.run(next_element))

[[b'You' b'use' b'PyTorch']
 [b'I' b'use' b'Tensorflow']]


In [58]:
dataset = tf.data.TextLineDataset("file.txt")
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
init_op = iterator.initializer

In [49]:
with tf.Session() as sess:
    # Initialize the iterator
    sess.run(init_op)
    print(sess.run(next_element))
    print(sess.run(next_element))
    # Move the iterator back to the beginning
    sess.run(init_op)
    print(sess.run(next_element))

b'I use Tensorflow'
b'You use PyTorch'
b'I use Tensorflow'


In [60]:
# images, labels = iterator.get_next()
# iterator_init_op = iterator.initializer

# inputs = {'images': images, 'labels': labels, 'iterator_init_op': iterator_init_op}

In [9]:
import sys
import os

# Copied from Pytorch src

IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif']


def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.
    Args:
        filename (string): path to a file
        extensions (iterable of strings): extensions to consider (lowercase)
    Returns:
        bool: True if the filename ends with one of given extensions
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in extensions)


def is_image_file(filename):
    """Checks if a file is an allowed image extension.
    Args:
        filename (string): path to a file
    Returns:
        bool: True if the filename ends with a known image extension
    """
    return has_file_allowed_extension(filename, IMG_EXTENSIONS)


def make_dataset(dir, class_to_idx, extensions):
    images = []
    dir = os.path.expanduser(dir)
    for target in sorted(class_to_idx.keys()):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue

        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if has_file_allowed_extension(fname, extensions):
                    path = os.path.join(root, fname)
                    item = (path, class_to_idx[target])
                    images.append(item)

    return images

def _find_classes(dir):
    """
    Finds the class folders in a dataset.
    Args:
        dir (string): Root directory path.
    Returns:
        tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
    Ensures:
        No class is a subdirectory of another.
    """
    if sys.version_info >= (3, 5):
        # Faster and available in Python 3.5 and above
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
    else:
        classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


dir = '/nfs1/datasets/imagenet_nfs1/train'

classes, class_to_idx = _find_classes(dir)
samples = make_dataset(dir, class_to_idx, IMG_EXTENSIONS)

In [11]:
type(classes[0])

str